In [229]:
import numpy as np
import pandas as pd

In [230]:
vg_qa = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/visual_genome/annotations/question_answers.json")

In [ ]:
vg_obj = pd.read_json("/Data2/Arun-UAV/NLP/vision_halu/visual_genome/annotations/objects.json")

In [ ]:
vg_obj["image_id"] = vg_obj["image_id"].apply(lambda x: str(int(x)))

In [ ]:
vg_obj["objects"].iloc[0]

[{'synsets': ['tree.n.01'],
  'h': 557,
  'object_id': 1058549,
  'merged_object_ids': [],
  'names': ['trees'],
  'w': 799,
  'y': 0,
  'x': 0},
 {'synsets': ['sidewalk.n.01'],
  'h': 290,
  'object_id': 1058534,
  'merged_object_ids': [5046],
  'names': ['sidewalk'],
  'w': 722,
  'y': 308,
  'x': 78},
 {'synsets': ['building.n.01'],
  'h': 538,
  'object_id': 1058508,
  'merged_object_ids': [],
  'names': ['building'],
  'w': 222,
  'y': 0,
  'x': 1},
 {'synsets': ['street.n.01'],
  'h': 258,
  'object_id': 1058539,
  'merged_object_ids': [3798578],
  'names': ['street'],
  'w': 359,
  'y': 283,
  'x': 439},
 {'synsets': ['wall.n.01'],
  'h': 535,
  'object_id': 1058543,
  'merged_object_ids': [],
  'names': ['wall'],
  'w': 135,
  'y': 1,
  'x': 0},
 {'synsets': ['tree.n.01'],
  'h': 360,
  'object_id': 1058545,
  'merged_object_ids': [],
  'names': ['tree'],
  'w': 476,
  'y': 0,
  'x': 178},
 {'synsets': ['shade.n.01'],
  'h': 189,
  'object_id': 5045,
  'merged_object_ids': [],


# Holoc object dataset collection

In [255]:

df_1 = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/vqa/tn_data.csv")
df_2 = pd.read_csv("/Data2/Arun-UAV/NLP/vision_halu/haloc/haloc_extension/vqa/tp_data.csv")
holoc_neg_data = pd.concat([df_1, df_2]).sample(12000)
holoc_neg_data = holoc_neg_data.drop_duplicates(subset=["question"])

In [256]:
holoc_neg_data.shape

(10006, 9)

In [258]:
holoc_neg_data["question"].nunique()

10006

In [259]:
from tqdm import tqdm
all_obj_info = []
obj_ids = []
for id in tqdm(holoc_neg_data["image_id"]):
    obj_df = pd.DataFrame(vg_obj[vg_obj["image_id"] == id.strip(".jpg")].iloc[0]["objects"])
    obj_df["names"] =  obj_df["names"].apply(lambda x: x[0])
    all_obj_info.append(obj_df.to_dict(orient="records"))
    obj_ids.append({j:i for i,j in zip(obj_df["object_id"] , obj_df["names"])})

100%|██████████| 10006/10006 [02:02<00:00, 81.93it/s]


In [260]:
holoc_neg_data["all_obj_info"] = all_obj_info
holoc_neg_data["obj_ids"] = obj_ids

In [261]:
holoc_neg_data.head(2)

,image_id,question,answer,hallucination_candidates,candidates,candidates_inx,image_path,question_id,hallucination_candidates_inx,all_obj_info,obj_ids
29691,2377030.jpg,Question: What is the woman on? Short answer:,book,['book'],[],NaN,/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,991c186e-0737-4a70-aec0-69a42cd945c3,"{'book': [(0, 4)]}","[{'synsets': ['radio_beam.n.01'], 'h': 39, 'ob...","{'beams': 568219, 'bed': 568229, 'bed room': 5..."
14647,2318398.jpg,Use the provided image to answer the question:...,girl,[],['girl'],"{'girl': [(0, 4)]}",/Data2/Arun-UAV/NLP/vision_halu/visual_genome/...,3048ca26-6f65-4279-b69d-35a64fcb89fa,NaN,"[{'synsets': [], 'h': 22, 'object_id': 4483469...","{'air': 4483469, 'antenna': 4055544, 'branches..."


In [262]:
holoc_neg_data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/evidence_head_train_datasets/holoc/vqa/holoc_sampled_10k.csv", index=False)

In [ ]:
image_processor = CLIPImageProcessor.from_pretrained(self.vision_tower_name)

In [ ]:
pd.DataFrame(vg_obj[vg_obj["image_id"] == id.strip(".jpg")].iloc[0]["objects"])

In [ ]:
def get_obj_words(res):
    words = [i["obj"]["name"] for i in eval(res)["object"]]
    return [{"word": i, 'h': 0, 'w': 0, 'y': 0,'x': 0} for i in words]

In [ ]:
fil_holoc_neg_data = holoc_neg_data[holoc_neg_data["annotations"].apply(lambda x: len(eval(x)["object"]) != 0)]
fil_holoc_neg_data["bb_info"] = fil_holoc_neg_data["annotations"].apply(get_obj_words)

fil_holoc_neg_data = fil_holoc_neg_data[["prompt", "image_id", "hallucinated_text", "bb_info"]]
fil_holoc_neg_data = fil_holoc_neg_data.rename(columns={"hallucinated_text": "answer"})
fil_holoc_neg_data["prompt"] = fil_holoc_neg_data["prompt"].apply(lambda x: x.replace("<image>", ""))


/tmp/ipykernel_2426391/2065153946.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  fil_holoc_neg_data["bb_info"] = fil_holoc_neg_data["annotations"].apply(get_obj_words)


In [ ]:
fil_holoc_neg_data.head(2)

,prompt,image_id,answer,bb_info
0,Explain the visual content of the image in gre...,2324272,A plumber in a blue shirt and black pants is s...,"[{'word': 'backpack', 'h': 0, 'w': 0, 'y': 0, ..."
2,What is this photo about?,2383098,In this image I see a horse which is of white ...,"[{'word': 'elephant', 'h': 0, 'w': 0, 'y': 0, ..."


In [ ]:
fil_holoc_neg_data.to_csv("/Data2/Arun-UAV/NLP/vision_halu/evidence_head_train_datasets/holoc/caption/obj_neg.csv", index=False)

In [ ]:
fil_holoc_neg_data.shape

(16931, 4)

In [264]:
from transformers import CLIPVisionModel, CLIPImageProcessor, CLIPVisionConfig
image_processor = CLIPImageProcessor.from_pretrained('openai/clip-vit-large-patch14-336')

/home/user/.conda/envs/shdm/lib/python3.10/site-packages/huggingface_hub/file_download.py:945: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
image_path = ""
Image.open(image_path).convert("RGB")
image_tensor = self.image_processor([image], return_tensors="pt")["pixel_values"]